<a href="https://colab.research.google.com/github/VinZCodz/machine-learning-works/blob/main/multi_variate_linear_regression/multi_variate_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic Math (Vector and Scalar)

In [ ]:
import numpy as np

a=np.array([1, 2, 3, 4])
b=np.array([8, 9, 1, 2])

c=a.dot(b)
print(f'Dot product of vector a with only 1 sample and 4 features DOT vector b with similiar shape and features= {c}')

a_mn=np.arange(6).reshape(2, 3)
print(f'a_mn=\n{a_mn}')
